# UCS Data Pipeline: Standardization & Normalization

**Dataset:** Union of Concerned Scientists (UCS) Satellite Database  
**Objective:** Prepare active satellite registry data for merger with SATCAT.

### **The Engineering Challenge**
The UCS database is human-maintained, leading to significant inconsistencies in categorical fields. To make this data machine-readable for our "Kessler Syndrome" analysis, we must implement a strict cleaning pipeline:
1.  **Ingestion & Sanitization:** Load raw data and neutralize whitespace/character artifacts.
2.  **Normalization:** Standardize "Country of Operator" and "Users" to ensure categorical consistency.
3.  **Physical Validation:** Enforce orbital mechanics constraints (e.g., Apogee vs. Perigee).
4.  **Mass Imputation:** Address missing values using the "ISS Exception" and grouped median fills.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Markdown, display

### **Stage 1: Ingestion & String Sanitization**
**The Problem:** Raw human-maintained data often contains hidden whitespace and character artifacts (e.g., " USA " vs "USA"), which causes silent failures during categorical grouping.

**The Solution:** Use a **lambda-based stripping** operation to strictly trim whitespace from all text-based columns and headers, ensuring a clean baseline for the pipeline.

In [2]:
ucs_sats_messy = pd.read_csv('../data/original/UCS-Satellite-Database 5-1-2023.csv')
text_cols = ucs_sats_messy.select_dtypes(['object']).columns

ucs_sats_messy[text_cols] = ucs_sats_messy[text_cols].apply(lambda x: x.str.strip())
ucs_sats_messy.columns = ucs_sats_messy.columns.str.strip()

### **Stage 1.1: Strategic Feature Selection**
**The Problem:** The raw UCS export contains numerous unpopulated placeholders (e.g., `Unnamed` columns) created by formatting artifacts in the original Excel file. These "Ghost Columns" inflate memory usage without adding information.

**The Solution:** Implement a **dynamic filter** to identify and drop all columns matching the `Unnamed` pattern, effectively sanitizing the dataframe structure.

In [3]:
unnamed_columns_dropped = [col for col in ucs_sats_messy.columns if 'Unnamed' in col]

if unnamed_columns_dropped:
    ucs_sats_messy.drop(columns=unnamed_columns_dropped, inplace=True)
    print(f"Dropped {len(unnamed_columns_dropped)} artifact columns (e.g., {unnamed_columns_dropped[0]}).")
else:
    print("No artifact columns found.")

Dropped 32 artifact columns (e.g., Unnamed: 28).


### **Stage 2: Enforcing Orbital Mechanics**
**The Problem:** Observational errors can result in physically impossible trajectories (Apogee < Perigee), and raw data often contains formatting artifacts (commas) that prevent numeric analysis.

**The Solution:**
* **Sanitize Numerics:** Remove string delimiters (commas) from `Perigee` and `Apogee`.
* **Physical Validation:** Implement a logical filter to ensure **Apogee (km) >= Perigee (km)**.

In [4]:
# Sanitize Numerics (Remove commas from strings)
ucs_sats_messy['Perigee (km)'] = ucs_sats_messy['Perigee (km)'].astype(str).str.replace(',', '', regex=False)
ucs_sats_messy['Apogee (km)'] = ucs_sats_messy['Apogee (km)'].astype(str).str.replace(',', '', regex=False)

# Convert to Float (Coerce errors to NaN)
ucs_sats_messy['Perigee (km)'] = pd.to_numeric(ucs_sats_messy['Perigee (km)'], errors='coerce')
ucs_sats_messy['Apogee (km)'] = pd.to_numeric(ucs_sats_messy['Apogee (km)'], errors='coerce')

# Drop missing values (We can't check physics if numbers are missing)
ucs_sats_messy.dropna(subset=['Perigee (km)', 'Apogee (km)'], inplace=True)

print("--- PRE-PATCH DIAGNOSTIC ---")
impossible_orbits = ucs_sats_messy[ucs_sats_messy['Apogee (km)'] < ucs_sats_messy['Perigee (km)']]
print(f"Satellites Violating Physics: {len(impossible_orbits)}")

if not impossible_orbits.empty:
    print("Violations Found:")
    print(impossible_orbits[['Name of Satellite, Alternate Names', 'Apogee (km)', 'Perigee (km)']].head(5))

# Fix known typo for Yaogan 35-5-1 (49.0 -> 499.0)
print("\n... Applying Manual Patch for Yaogan 35-5-1 ...\n")
typo_mask = ucs_sats_messy['Name of Satellite, Alternate Names'] == 'Yaogan 35-5-1'
ucs_sats_messy.loc[typo_mask, 'Apogee (km)'] = 499.0

print("--- POST-PATCH DIAGNOSTIC ---")
impossible_orbits_after = ucs_sats_messy[ucs_sats_messy['Apogee (km)'] < ucs_sats_messy['Perigee (km)']]
print(f"Satellites Violating Physics: {len(impossible_orbits_after)}")

if impossible_orbits_after.empty:
    print("✅ SUCCESS: All physics violations resolved.")

# Only keeps valid rows. Since we fixed the error, we lose 0 satellites here.
ucs_sats_messy = ucs_sats_messy[ucs_sats_messy['Apogee (km)'] >= ucs_sats_messy['Perigee (km)']]

print(f"\nTotal Satellites Retained: {len(ucs_sats_messy)}")

--- PRE-PATCH DIAGNOSTIC ---
Satellites Violating Physics: 1
Violations Found:
     Name of Satellite, Alternate Names  Apogee (km)  Perigee (km)
7473                      Yaogan 35-5-1         49.0         493.0

... Applying Manual Patch for Yaogan 35-5-1 ...

--- POST-PATCH DIAGNOSTIC ---
Satellites Violating Physics: 0
✅ SUCCESS: All physics violations resolved.

Total Satellites Retained: 7553


### **Stage 3: Metadata Decoupling (Source Preservation)**
**The Problem:** Carrying extensive "Comments" and "Source" columns creates "Wide Data" that is inefficient for large-scale physics modeling.

**The Solution:**
* **Source Archive:** Extract and save metadata into a secondary file (`ucs_dropped.csv`).
* **Relational Key:** Retain the `norad_id` as a primary key to allow for future re-integration of this context if needed.

In [5]:
# We define this list so we never accidentally drop something we didn't save
archived_columns = [
    'Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2', 
    'Source.3', 'Source.4', 'Source.5', 'Source.6', 'Comments'
]

# We grab exactly the columns defined above
sources = ucs_sats_messy[archived_columns].copy()

# We insert the 'norad_id' now so it matches the future clean dataset
sources.insert(0, 'norad_id', ucs_sats_messy['NORAD Number'])

# 4. Save metadata into a secondary file, ucs_dropped.csv
sources = sources.sort_values(by='norad_id')
sources.to_csv('../data/clean/ucs_dropped.csv', index=False)
print(f"Archived {sources.shape[1]} columns to 'ucs_dropped.csv'")

# Use the archived_columns varible we defined earlier to drop only what weve already exported.
ucs_sats_messy.drop(columns=archived_columns, inplace=True)

print(f"Dropped {len(archived_columns)} columns from active memory.")

Archived 10 columns to 'ucs_dropped.csv'
Dropped 9 columns from active memory.


### **Stage 3.1: Categorical and Temporal Sanitization**

**The Problem:** Mixed-case strings, trailing whitespaces, and string-formatted dates create "duplicate" categories (e.g., "SpaceX" vs "SpaceX ") and prevent accurate time-series analysis.

**The Solution:**
* **Temporal Conversion:** Parse Date of Launch into standard datetime objects and remove the negligible number of rows missing this data to ensure a reliable timeline.
* **Case & Whitespace Normalization:** Force Class of Orbit to uppercase and strip() all major categorical columns to ensure unique, clean labels for grouping.
* **Logical Metadata Infill:** Perform early-stage cleanup on Contractor for vertically integrated companies and apply standard fallbacks for missing launch metadata to achieve 100% categorical density.

In [6]:
# Temporal Conversion
# Convert to datetime and drop the 1-2 rows with missing dates to ensure timeline integrity
ucs_sats_messy['Date of Launch'] = pd.to_datetime(ucs_sats_messy['Date of Launch'], errors='coerce')
ucs_sats_messy = ucs_sats_messy.dropna(subset=['Date of Launch'])

# Case and Whitespace Normalization
# Standardize the primary orbit class to uppercase
ucs_sats_messy['Class of Orbit'] = ucs_sats_messy['Class of Orbit'].str.upper()

# List of columns that need whitespace scrubbing
scrub_cols = [
    'Operator/Owner', 'Contractor', 'Launch Site', 
    'Launch Vehicle', 'Purpose', 'Type of Orbit'
]

for col in scrub_cols:
    if col in ucs_sats_messy.columns:
        ucs_sats_messy[col] = ucs_sats_messy[col].str.strip()

# Targeted Metadata Infill
# Fill SpaceX contractor gaps where they are clearly the builder
ucs_sats_messy.loc[(ucs_sats_messy['Operator/Owner'] == 'SpaceX') & 
                   (ucs_sats_messy['Contractor'].isna()), 'Contractor'] = 'SpaceX'

# General fallback for remaining metadata gaps
ucs_sats_messy['Contractor'] = ucs_sats_messy['Contractor'].fillna('Unknown/Multiple')
ucs_sats_messy['Launch Site'] = ucs_sats_messy['Launch Site'].fillna('Unknown Site')
ucs_sats_messy['Launch Vehicle'] = ucs_sats_messy['Launch Vehicle'].fillna('Unknown Vehicle')

print(f"Sanitization Complete: {len(ucs_sats_messy)} satellites standardized.")
print(f"Remaining Nulls in Metadata: {ucs_sats_messy[scrub_cols].isnull().sum().sum()}")

Sanitization Complete: 7551 satellites standardized.
Remaining Nulls in Metadata: 651


### **Stage 3.2: Universal Numeric Sanitization (The Physics 10)**

**The Problem:** High-fidelity physics modeling requires strict numeric types. However, fields like `Launch Mass`, `Period`, and `Perigee` often contain human-entered string artifacts—such as commas in "1,200" or non-numeric notes—that force Pandas to treat the entire column as an `object` (string).

**The Solution:**
* **Neutralize Delimiters:** Implement a universal string-replacement loop to strip commas across all 10 key physics columns.
* **Type Enforcement:** Utilize `pd.to_numeric` with `errors='coerce'`. This gracefully handles irregular entries (e.g., "15 years" or "~500") by converting them to `NaN`, ensuring the data is mathematically valid for Stage 4 calculations.
* **Immediate Diagnostic:** Execute a type-verification audit to confirm that every physics field has transitioned to `float64`.

**Impact:** This ensures that the "Physics Reconstruction Engine" in the next stage has a stable, purely numeric foundation to work from.

In [7]:
# The list of columns that must be numeric
columns_to_sanitize = [
    'Launch Mass (kg.)', 
    'Dry Mass (kg.)', 
    'Power (watts)',
    'Period (minutes)',
    'Expected Lifetime (yrs.)',
    'Perigee (km)',
    'Apogee (km)',
    'Eccentricity',
    'Inclination (degrees)',
    'Longitude of GEO (degrees)'
]

print(f"Sanitizing {len(columns_to_sanitize)} physics columns...")

# THE CLEANING LOOP
for col in columns_to_sanitize:
    if col in ucs_sats_messy.columns:
        # Force to string, remove commas, and coerce to float
        ucs_sats_messy[col] = ucs_sats_messy[col].astype(str).str.replace(',', '', regex=False)
        ucs_sats_messy[col] = pd.to_numeric(ucs_sats_messy[col], errors='coerce')

print("Numeric Sanitization Complete.")

# IMMEDIATE VERIFICATION (Diagnostic)
print(f"\n{'Column (Original Name)':<30} | {'Current Type':<15} | {'Sample Value':<15} | {'Count'}")
print("-" * 75)

for col in columns_to_sanitize:
    if col in ucs_sats_messy.columns:
        dtype = str(ucs_sats_messy[col].dtype)
        # Get a sample value (first non-null)
        sample = ucs_sats_messy[col].dropna().iloc[0] if not ucs_sats_messy[col].dropna().empty else "Empty"
        count = ucs_sats_messy[col].count()
        # VISUAL ALARM: If it's an object, mark it with '!!!'
        # We want to see 'float64' or 'int64'. 'object' is a failure.
        status_marker = "!!!" if 'object' in dtype else ""
        
        print(f"{col:<30} | {dtype:<15} | {sample:<15} | {count} {status_marker}")
    else:
        print(f"{col:<30} | NOT FOUND")

Sanitizing 10 physics columns...
Numeric Sanitization Complete.

Column (Original Name)         | Current Type    | Sample Value    | Count
---------------------------------------------------------------------------
Launch Mass (kg.)              | float64         | 22.0            | 7308 
Dry Mass (kg.)                 | float64         | 4.0             | 758 
Power (watts)                  | float64         | 4.5             | 557 
Period (minutes)               | float64         | 96.08           | 7498 
Expected Lifetime (yrs.)       | float64         | 0.5             | 5449 
Perigee (km)                   | float64         | 566.0           | 7551 
Apogee (km)                    | float64         | 576.0           | 7551 
Eccentricity                   | float64         | 0.00151         | 7547 
Inclination (degrees)          | float64         | 36.9            | 7550 
Longitude of GEO (degrees)     | float64         | 0.0             | 7549 


### **Stage 3.3: Primary Identifier Sanitization**

**The Problem:** Leading spaces in COSPAR Number and decimal artifacts in NORAD Number (e.g., "25544.0") will cause merge failures when connecting to the SATCAT.
**The Solution:**
* **String Scrubbing:** Strip whitespace and force COSPAR to uppercase.
* **Numeric Normalization:** Force NORAD Number through a numeric conversion to strip decimals before converting to a clean string. 
* **Null Recovery:** Re-convert "NAN" strings back to proper np.nan objects.

In [8]:
print("Sanitizing primary identifiers...")

# Clean COSPAR IDs (Removing leading spaces and normalizing case)
ucs_sats_messy['COSPAR Number'] = ucs_sats_messy['COSPAR Number'].astype(str).str.strip().str.upper()

# Clean NORAD IDs (Safety conversion to remove decimal artifacts)
# We coerce to numeric first to turn "25544.0" into 25544
ucs_sats_messy['NORAD Number'] = pd.to_numeric(ucs_sats_messy['NORAD Number'], errors='coerce')

# Drop rows where NORAD ID is missing (we cannot merge satellites without an ID)
ucs_sats_messy = ucs_sats_messy.dropna(subset=['NORAD Number'])

# Convert to clean integer strings (25544.0 -> 25544 -> "25544")
ucs_sats_messy['NORAD Number'] = ucs_sats_messy['NORAD Number'].astype(int).astype(str)

# Re-neutralize 'nan' strings for COSPAR
ucs_sats_messy['COSPAR Number'] = ucs_sats_messy['COSPAR Number'].replace('NAN', np.nan)

print(f"Identifier Sanitization Complete: {len(ucs_sats_messy)} records ready for merging.")

Sanitizing primary identifiers...
Identifier Sanitization Complete: 7551 records ready for merging.


### **Verification: Intermediate Pipeline Audit (The Quality Gate)**

**Objective:** This audit serves as a critical "Quality Gate" to verify the structural integrity of the dataframe before it enters the Stage 4 Physics Reconstruction Engine. 

**Validation Targets:**
1.  **Relational Integrity:** Confirm `Source` metadata was successfully decoupled and archived to `ucs_dropped.csv`.
2.  **Memory Optimization:** Ensure "Ghost Columns" and non-essential strings are purged to maintain a "Lean" modeling environment.
3.  **Type Enforcement:** Confirm that mathematical fields (Perigee, Apogee, Mass) have been successfully sanitized of string artifacts and are ready for calculation.

In [9]:
# Structural Check: Ghost & Metadata Columns
remaining_unnamed = [col for col in ucs_sats_messy.columns if 'Unnamed' in col]
metadata_to_check = [
    'Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2', 
    'Source.3', 'Source.4', 'Source.5', 'Source.6', 'Comments'
]
remaining_metadata = [col for col in metadata_to_check if col in ucs_sats_messy.columns]

# Type Check: Comprehensive Physics Scan
# We check a subset of critical fields to ensure Stage 3.2 worked across the board
physics_cols = ['Perigee (km)', 'Apogee (km)', 'Launch Mass (kg.)']
physics_status = all(pd.api.types.is_numeric_dtype(ucs_sats_messy[col]) for col in physics_cols)

# Relational Check: Identifier Whitespace
# We verify Stage 3.3 by checking if any NORAD ID still has a space
space_check = ucs_sats_messy['NORAD Number'].str.contains(' ').any()

print(f"{'--- INTERMEDIATE PIPELINE AUDIT ---':^45}")
print(f"{'CHECK':<25} | {'STATUS'}")
print("-" * 45)

# Report Ghost Columns
ghost_status = "✅ CLEAN" if not remaining_unnamed else f"❌ FOUND: {len(remaining_unnamed)}"
print(f"{'Ghost Column Purge':<25} | {ghost_status}")

# Report Metadata Purge
meta_status = "✅ SUCCESS" if not remaining_metadata else "❌ FAILED"
print(f"{'Metadata Archive':<25} | {meta_status}")

# Report Physics Typing
type_status = "✅ NUMERIC" if physics_status else "❌ STRING ERROR"
print(f"{'Physics Type Enforcement':<25} | {type_status}")

# Report ID Sanitization (Whitespace check)
id_status = "✅ WHITESPACE-FREE" if not space_check else "❌ SPACE DETECTED"
print(f"{'Identifier Sanitization':<25} | {id_status}")

print("-" * 45)
print(f"Final Pre-Imputation Count: {len(ucs_sats_messy):,} satellites")

# Final logic gate: Stop the user if something is fundamentally broken
if not physics_status or space_check:
    print("\n⚠️  WARNING: Quality Gate failed. Review Stage 3 before proceeding.")
else:
    print("\n🚀 PASS: Dataset is officially Physics-Ready.")

     --- INTERMEDIATE PIPELINE AUDIT ---     
CHECK                     | STATUS
---------------------------------------------
Ghost Column Purge        | ✅ CLEAN
Metadata Archive          | ✅ SUCCESS
Physics Type Enforcement  | ✅ NUMERIC
Identifier Sanitization   | ✅ WHITESPACE-FREE
---------------------------------------------
Final Pre-Imputation Count: 7,551 satellites

🚀 PASS: Dataset is officially Physics-Ready.


### **Stage 4.1: Addressing the Physics Transparency Gap (Mass & Power)**
**The Problem:** Critical physical properties (`Launch Mass`, `Dry Mass`, `Power`) are missing for significant portions of the registry. Deleting these rows would hide risk; leaving them empty breaks kinetic modeling.

**The Solution:**
* **The "White Whale" Exception:** Manually set the **ISS** mass (450,000 kg) to prevent it from skewing statistical medians.
* **Grouped Median Imputation:** Fill `Launch Mass` and `Power` using the median of satellites with similar **Orbit** and **Purpose**.
* **Physics-Informed Ratio:** Derive `Dry Mass` by calculating the typical *Dry-to-Wet Ratio* for each orbit class and applying it to the satellite's launch mass.

In [10]:
# THE ISS EXCEPTION (THE "WHITE WHALE")
# We manually set the station mass first because it is a unique outlier.
# We use '25544' as a string to match the Stage 3.3 sanitization and create our mask to make things easier.
iss_mask = ucs_sats_messy['NORAD Number'] == '25544'

ucs_sats_messy.loc[iss_mask, 'Launch Mass (kg.)'] = 450000
ucs_sats_messy.loc[iss_mask, 'Power (watts)'] = 84000
ucs_sats_messy.loc[iss_mask, 'Dry Mass (kg.)'] = 420000

print(f"ISS (NORAD 25544) manually patched: 450,000kg Mass | 420,000kg Dry | 84kW Power")

# IMPUTE LAUNCH MASS & POWER (GROUPED MEDIANS)
# Logic: Satellites with the same mission (Purpose) in the same region (Class of Orbit) 
# usually share similar chassis types (e.g., Starlink, OneWeb).
print("Imputing Launch Mass & Power via Grouped Medians...")
fill_cols = ['Launch Mass (kg.)', 'Power (watts)']

for col in fill_cols:
    # Calculate medians based on the specific peer group
    medians = ucs_sats_messy.groupby(['Class of Orbit', 'Purpose'])[col].transform('median')
    ucs_sats_messy[col] = ucs_sats_messy[col].fillna(medians)

# Orbit Fallback for Mass
orbit_medians_mass = ucs_sats_messy.groupby('Class of Orbit')['Launch Mass (kg.)'].transform('median')
ucs_sats_messy['Launch Mass (kg.)'] = ucs_sats_messy['Launch Mass (kg.)'].fillna(orbit_medians_mass)

# Orbit Fallback for Power
orbit_medians_pwr = ucs_sats_messy.groupby('Class of Orbit')['Power (watts)'].transform('median')
ucs_sats_messy['Power (watts)'] = ucs_sats_messy['Power (watts)'].fillna(orbit_medians_pwr)

# Global Fallback for Mass (The Absolute Safety Net)
global_mass_median = ucs_sats_messy['Launch Mass (kg.)'].median()
ucs_sats_messy['Launch Mass (kg.)'] = ucs_sats_messy['Launch Mass (kg.)'].fillna(global_mass_median)

# Global Fallback for Power (The Absolute Safety Net)
global_power_median = ucs_sats_messy['Power (watts)'].median()
ucs_sats_messy['Power (watts)'] = ucs_sats_messy['Power (watts)'].fillna(global_power_median)

# IMPUTE DRY MASS (RATIO-DERIVED)
# We cannot use simple medians for Dry Mass because a 1kg CubeSat shouldn't 
# receive a 1,000kg median mass. We use the structural ratio instead.
print("Imputing Dry Mass via Orbit-Specific Mass Ratios...")

# Calculate existing ratios (Dry Mass / Launch Mass)
ucs_sats_messy['mass_ratio'] = ucs_sats_messy['Dry Mass (kg.)'] / ucs_sats_messy['Launch Mass (kg.)']

# Get the median ratio for each orbit (e.g., LEO sats vs. massive GEO commsats)
ratio_medians = ucs_sats_messy.groupby('Class of Orbit')['mass_ratio'].transform('median')
ucs_sats_messy['mass_ratio'] = ucs_sats_messy['mass_ratio'].fillna(ratio_medians)

# Apply the ratio to the specific satellite's actual Launch Mass
estimated_dry_mass = ucs_sats_messy['Launch Mass (kg.)'] * ucs_sats_messy['mass_ratio']
ucs_sats_messy['Dry Mass (kg.)'] = ucs_sats_messy['Dry Mass (kg.)'].fillna(estimated_dry_mass)

# Drop the temporary ratio column
ucs_sats_messy.drop(columns=['mass_ratio'], inplace=True)

# 5. FINAL PHYSICS AUDIT
print("\n--- Physics Gap Audit (Remaining Missing Values) ---")
print(f"Launch Mass: {ucs_sats_messy['Launch Mass (kg.)'].isnull().sum()}")
print(f"Dry Mass:    {ucs_sats_messy['Dry Mass (kg.)'].isnull().sum()}")
print(f"Power:       {ucs_sats_messy['Power (watts)'].isnull().sum()}")

# Visual Verification of the ISS
display(ucs_sats_messy[iss_mask][['Name of Satellite, Alternate Names', 'Launch Mass (kg.)', 'Power (watts)']])

ISS (NORAD 25544) manually patched: 450,000kg Mass | 420,000kg Dry | 84kW Power
Imputing Launch Mass & Power via Grouped Medians...
Imputing Dry Mass via Orbit-Specific Mass Ratios...

--- Physics Gap Audit (Remaining Missing Values) ---
Launch Mass: 0
Dry Mass:    0
Power:       0


,"Name of Satellite, Alternate Names",Launch Mass (kg.),Power (watts)
1236,International Space Station (ISS [first elemen...,450000.0,84000.0


### **Stage 4.2: Orbital & Lifecycle Sweep (The Final Gaps)**

**The Problem:** Secondary gaps in orbital elements (`Period`) and operational data (`Expected Lifetime`) prevent a total kinetic and temporal model.

**The Solution:**
1.  **Keplerian Derivation:** Use Kepler’s Third Law to mathematically calculate missing **Orbital Periods** from existing Perigee/Apogee data.
2.  **Lifecycle Imputation:** Fill missing **Expected Lifetimes** using medians grouped by `Class of Orbit`.
3.  **The "Dense" Registry:** Apply a final median sweep to ensure all 10 physics columns have 0 missing values.

In [11]:
print("Executing Final Physics Sweep...")

# Kepler's Third Law (Calculating Period from Altitude)
# Formula: T = 2 * pi * sqrt(a^3 / mu)
# semi-major axis (a) = Earth_Radius + (Perigee + Apogee) / 2
earth_radius = 6378.137
mu = 398600.4418 # Earth's gravitational parameter (km^3/s^2)

def calculate_period(row):
    # Only calculate if Period is missing but we have altitudes
    if pd.isna(row['Period (minutes)']) and not pd.isna(row['Perigee (km)']) and not pd.isna(row['Apogee (km)']):
        # a is the semi-major axis in km
        a = earth_radius + ((row['Perigee (km)'] + row['Apogee (km)']) / 2)
        period_seconds = 2 * np.pi * np.sqrt(a**3 / mu)
        return period_seconds / 60
    return row['Period (minutes)']

# Apply the physics derivation
ucs_sats_messy['Period (minutes)'] = ucs_sats_messy.apply(calculate_period, axis=1)

# Impute Remaining Elements & Lifetime (Grouped Medians)
# This handles the ~2,100 missing Expected Lifetime values
sweep_cols = [
    'Expected Lifetime (yrs.)', 'Period (minutes)', 
    'Inclination (degrees)', 'Eccentricity', 
    'Perigee (km)', 'Apogee (km)', 'Longitude of GEO (degrees)'
]

for col in sweep_cols:
    if col in ucs_sats_messy.columns:
        # We use Orbit Class to group because LEO/GEO physics vary wildly
        orbit_medians = ucs_sats_messy.groupby('Class of Orbit')[col].transform('median')
        ucs_sats_messy[col] = ucs_sats_messy[col].fillna(orbit_medians)

# Global Fallback (Final Safety Net)
ucs_sats_messy['Expected Lifetime (yrs.)'] = ucs_sats_messy['Expected Lifetime (yrs.)'].fillna(ucs_sats_messy['Expected Lifetime (yrs.)'].median())

# FINAL COVERAGE VERIFICATION
print(f"\n{'Column':<30} | {'Status'}")
print("-" * 50)
all_physics = sweep_cols + ['Launch Mass (kg.)', 'Dry Mass (kg.)', 'Power (watts)']
for col in all_physics:
    missing = ucs_sats_messy[col].isnull().sum()
    print(f"{col:<30} | {'✅ COMPLETE' if missing == 0 else f'❌ {missing} MISSING'}")

Executing Final Physics Sweep...

Column                         | Status
--------------------------------------------------
Expected Lifetime (yrs.)       | ✅ COMPLETE
Period (minutes)               | ✅ COMPLETE
Inclination (degrees)          | ✅ COMPLETE
Eccentricity                   | ✅ COMPLETE
Perigee (km)                   | ✅ COMPLETE
Apogee (km)                    | ✅ COMPLETE
Longitude of GEO (degrees)     | ✅ COMPLETE
Launch Mass (kg.)              | ✅ COMPLETE
Dry Mass (kg.)                 | ✅ COMPLETE
Power (watts)                  | ✅ COMPLETE


### **Stage 5: Schema Alignment (Renaming & Type Finalization)**

**The Problem:** Raw UCS headers (e.g., `Name of Satellite, Alternate Names`) are too verbose for efficient coding and contain spaces/parentheses that can break certain SQL or Python operations.

**The Solution:** Implement a global **Renaming Schema** to transition the dataset into a strict **snake_case** format. 
1. **Primary Key Alignment:** Rename `NORAD Number` to `norad_id` to match the SATCAT pipeline.
2. **Physics Standardizing:** Shorten mass and power headers for programmatic speed.
3. **Identifier Cleaning:** Finalize `COSPAR Number` as `cospar_id`.

In [12]:
# Create rename mapping
column_mapping = {
    'Name of Satellite, Alternate Names': 'satellite_name',
    'Current Official Name of Satellite': 'official_name',
    'Country/Org of UN Registry': 'un_registry',
    'Country of Operator/Owner': 'country_operator',
    'Operator/Owner': 'owner',
    'Users': 'users',
    'Purpose': 'purpose',
    'Class of Orbit': 'orbit_class',
    'Type of Orbit': 'orbit_type',
    'Longitude of GEO (degrees)': 'geo_longitude',
    'Perigee (km)': 'perigee_km',
    'Apogee (km)': 'apogee_km',
    'Eccentricity': 'eccentricity',
    'Inclination (degrees)': 'inclination_degrees',
    'Period (minutes)': 'period_minutes',
    'Launch Mass (kg.)': 'launch_mass_kg',
    'Date of Launch': 'launch_date',
    'Expected Lifetime (yrs.)': 'lifetime_years',
    'Contractor': 'contractor',
    'Country of Contractor': 'contractor_country',
    'Launch Site': 'launch_site',
    'Launch Vehicle': 'launch_vehicle',
    'COSPAR Number': 'cospar_id',
    'NORAD Number': 'norad_id',
    'Detailed Purpose': 'detailed_purpose',
    'Dry Mass (kg.)': 'dry_mass_kg',
    'Power (watts)': 'power_watts'
}

# Apply the Rename
ucs_sats_messy.rename(columns=column_mapping, inplace=True)

# Verification Check: Remaining messy headers
messy_headers = [col for col in ucs_sats_messy.columns if ' ' in col or '(' in col]

print(f"--- Schema Finalization Report ---")
print(f"Total Columns Standardized: {len(ucs_sats_messy.columns)}")
print(f"Messy Headers Remaining:    {'None (Full Clean)' if not messy_headers else messy_headers}")
print(f"Primary Merge Key:          { 'norad_id' in ucs_sats_messy.columns}")

--- Schema Finalization Report ---
Total Columns Standardized: 27
Messy Headers Remaining:    None (Full Clean)
Primary Merge Key:          True


### **Stage 6: Feature Engineering (Boolean Flags & Mission Standardization)**
**The Problem:** The `users` column contains complex multi-stakeholder strings (e.g., "Government/Commercial/Military"), and the `purpose` column contains inconsistent terminology (e.g., "Earth Science" vs. "Earth Observation").

**The Solution:**
* **Boolean Flags:** Decompose the `users` column into binary indicators (`is_commercial`, `is_government`, `is_military`, `is_civil`) to enable precise sector-based querying.
* **Mission Standardization:** Map diverse mission descriptions to a controlled vocabulary (e.g., Mapping "Surveillance" and "Meteorological" to **"Earth Observation"**).

In [13]:
# Create User Boolean Flags (The "Democratization" Columns)
# These flags allow queries like: "Show me Civil satellites with NO Government involvement"
ucs_sats_messy['is_commercial'] = ucs_sats_messy['users'].str.contains('Commercial', case=False, na=False).astype(int)
ucs_sats_messy['is_government'] = ucs_sats_messy['users'].str.contains('Government', case=False, na=False).astype(int)
ucs_sats_messy['is_military'] = ucs_sats_messy['users'].str.contains('Military', case=False, na=False).astype(int)
ucs_sats_messy['is_civil'] = ucs_sats_messy['users'].str.contains('Civil', case=False, na=False).astype(int)

# Standardize Primary Purpose (The "Mission")
def standardize_purpose(text):
    if pd.isna(text) or text == 'Unknown':
        return 'Unknown'
    
    # Take the first primary term if there are multiple (e.g. "Comms/Nav")
    primary = text.split('/')[0].strip()
    
    mapping = {
        'Earth Science': 'Earth Observation',
        'Meteorological': 'Earth Observation',
        'Surveillance': 'Earth Observation',
        'Earth': 'Earth Observation',
        'Earth/Space Observation': 'Earth Observation',
        'Space Observation': 'Space Science',
        'Technology Demonstration': 'Technology Development',
        'Mission Extension Technology': 'Technology Development',
        'Platform': 'Technology Development',
        'Satellite Positioning': 'Navigation',
        'Navigation': 'Navigation',
        'Communications': 'Communications',
        'Space Science': 'Space Science',
        'Educational': 'Educational'
    }
    return mapping.get(primary, primary)

ucs_sats_messy['primary_purpose'] = ucs_sats_messy['purpose'].apply(standardize_purpose)

# Logical Reordering (Move primary_purpose next to purpose for easy checking)
cols = list(ucs_sats_messy.columns)
cols.remove('primary_purpose')
target_index = cols.index('purpose')
cols.insert(target_index + 1, 'primary_purpose')
ucs_sats_messy = ucs_sats_messy[cols]

print("\n--- Mission Standardization Audit (Change Detection) ---")
columns_to_show = ['satellite_name', 'purpose', 'primary_purpose']
diff_view = ucs_sats_messy[ucs_sats_messy['purpose'] != ucs_sats_messy['primary_purpose']][columns_to_show]

if not diff_view.empty:
    print(f"Success: Standardized {len(diff_view)} complex mission labels into controlled vocabulary.")
    display(diff_view.head(10))
else:
    print("Verification: No complex labels found.")


--- Mission Standardization Audit (Change Detection) ---
Success: Standardized 324 complex mission labels into controlled vocabulary.


,satellite_name,purpose,primary_purpose
12,ADLER-2,Earth Science,Earth Observation
51,ALE-2 (Astro Live Experiences-2),Earth Science,Earth Observation
75,ANDESITE Mule (Ad-Hoc Network Demonstration fo...,Space Science/Technology Demonstration,Space Science
83,AprizeSat 1 (LatinSat-C),Communications/Technology Development,Communications
84,AprizeSat 10 (exactView-13),Communications/Maritime Tracking,Communications
85,AprizeSat 2 (LatinSat-D),Communications/Technology Development,Communications
86,AprizeSat 3,Communications/Maritime Tracking,Communications
87,AprizeSat 4,Communications/Maritime Tracking,Communications
90,AprizeSat 8 (exactView-12),Communications/Maritime Tracking,Communications
91,AprizeSat 9 (exactView-11),Communications/Maritime Tracking,Communications


### **Stage 6.1: Orbit Class Standardization**
**The Problem:** The `orbit_class` column contains synonymous but inconsistent labels (e.g., "Low Earth Orbit" vs "LEO"). 

**The Solution:** Implement a mapping dictionary to consolidate all orbital regimes into four standardized categories: **LEO, MEO, GEO,** and **Elliptical**. This ensures compatibility with the SATCAT classification logic used in the next phase of the pipeline.

###6.2

In [14]:
# Final Categorical Gap Check
metadata_cols = ['owner', 'contractor', 'launch_site', 'launch_vehicle']
print("--- Metadata Gap Audit ---")
print(ucs_sats_messy[metadata_cols].isnull().sum())

--- Metadata Gap Audit ---
owner             0
contractor        0
launch_site       0
launch_vehicle    0
dtype: int64


In [15]:
# Standardize Orbit Class (The Region)
ucs_sats_messy['orbit_class'] = ucs_sats_messy['orbit_class'].str.upper()

orbit_class_map = {
    'LEO': 'LEO',
    'GEO': 'GEO',
    'MEO': 'MEO',
    'ELLIPTICAL': 'Elliptical',
    'Elliptical': 'Elliptical'
}

# 2. Standardize Orbit Type (The Geometry)
# We'll group these into Polar, Non-Polar, and Eccentric
orbit_type_map = {
    'Non-Polar Inclined': 'Inclined',
    'Sun-Synchronous': 'Polar',
    'Polar': 'Polar',
    'Equatorial': 'Equatorial',
    'Molniya': 'Eccentric',
    'Deep Highly Eccentric': 'Eccentric',
    'Elliptical': 'Eccentric',
    'Sun-Synchronous near polar': 'Polar',
    'Cislunar': 'Eccentric',
    'Retrograde': 'Inclined'
}

# Apply mappings with .str.strip() to catch any hidden spaces
ucs_sats_messy['orbit_class'] = ucs_sats_messy['orbit_class'].str.strip().map(orbit_class_map).fillna('Unknown')
ucs_sats_messy['orbit_type'] = ucs_sats_messy['orbit_type'].str.strip().map(orbit_type_map).fillna('Other/Misc')

print("--- Final Distribution Post-Fix ---")
print(ucs_sats_messy['orbit_class'].value_counts())

--- Final Distribution Post-Fix ---
orbit_class
LEO           6759
GEO            590
MEO            143
Elliptical      59
Name: count, dtype: int64


### **Stage 7: Pipeline Serialization & Executive Summary**
**Objective:** Finalize the active population for export.

We have successfully addressed two critical data gaps:
1. **The "Mass Transparency Gap":** Addressed via the **ISS Exception**, **Grouped Median Imputation**, and **Physics-Informed Ratios**, making this the high-fidelity mass reference for collision models.
2. **The "Metadata Consistency Gap":** Addressed via **Boolean Sector Flags** (e.g., `is_military`) and **Mission Standardization**, transforming raw text into machine-readable categories.

**Outcome:** This dataset is now normalized, validated, and ready for export.

In [16]:
output_path = '../data/clean/ucs_cleaned.csv'

# Calculate Sector Metrics
total_rows = len(ucs_sats_messy)
comm_count = ucs_sats_messy['is_commercial'].sum()
mil_count  = ucs_sats_messy['is_military'].sum()
gov_count  = ucs_sats_messy['is_government'].sum()
civ_count  = ucs_sats_messy['is_civil'].sum()

# Congestion Metrics (The Polar Alert)
# Focusing on LEO for congestion analysis
leo_mask = ucs_sats_messy['orbit_class'] == 'LEO'
polar_leo_count = ucs_sats_messy[leo_mask & (ucs_sats_messy['orbit_type'] == 'Polar')].shape[0]
polar_share = (polar_leo_count / ucs_sats_messy[leo_mask].shape[0]) if total_rows > 0 else 0

# Calculate Mission Metrics
top_missions = ucs_sats_messy['primary_purpose'].value_counts().head(3)
m1_n, m1_c = top_missions.index[0], top_missions.values[0]
m2_n, m2_c = top_missions.index[1], top_missions.values[1]
m3_n, m3_c = top_missions.index[2], top_missions.values[2]

# Comprehensive Data Quality Audit
# Including geo_longitude to ensure 100% density across the physics suite
physics_features = [
    'launch_mass_kg', 'dry_mass_kg', 'power_watts', 
    'period_minutes', 'lifetime_years', 'perigee_km', 
    'apogee_km', 'eccentricity', 'inclination_degrees',
    'geo_longitude'
]

# Generate the report
report = f"""
### **UCS Pipeline Completion Report**
**Total Active Registry:** {total_rows:,} satellites

#### **Sector Composition**
| Sector | Count | Share |
| :--- | :--- | :--- |
| **Commercial** | {comm_count:,} | {comm_count/total_rows:.1%} |
| **Military** | {mil_count:,} | {mil_count/total_rows:.1%} |
| **Government** | {gov_count:,} | {gov_count/total_rows:.1%} |
| **Civil** | {civ_count:,} | {civ_count/total_rows:.1%} |
| **Polar Congestion (LEO)** | {polar_leo_count:,} | {polar_share:.1%} of LEO ⚠️ |

#### **Primary Mission Breakdown**
| Top Mission | Count | Share |
| :--- | :--- | :--- |
| **1. {m1_n}** | {m1_c:,} | {m1_c/total_rows:.1%} |
| **2. {m2_n}** | {m2_c:,} | {m2_c/total_rows:.1%} |
| **3. {m3_n}** | {m3_c:,} | {m3_c/total_rows:.1%} |

#### **Data Quality & Imputation Success**
| Feature | Completeness | Method | Status |
| :--- | :--- | :--- | :--- |
"""

for feature in physics_features:
    coverage = ucs_sats_messy[feature].notna().mean()
    method = "Calculated/Imputed" if coverage == 1.0 else "Incomplete"
    report += f"| **{feature.replace('_', ' ').title()}** | **{coverage:.1%}** | {method} | ✅ SUCCESS |\n"

report += f"\n💾 **File Saved:** `{output_path}`"

display(Markdown(report))
ucs_sats_messy.to_csv(output_path, index=False)


### **UCS Pipeline Completion Report**
**Total Active Registry:** 7,551 satellites

#### **Sector Composition**
| Sector | Count | Share |
| :--- | :--- | :--- |
| **Commercial** | 6,260 | 82.9% |
| **Military** | 613 | 8.1% |
| **Government** | 762 | 10.1% |
| **Civil** | 219 | 2.9% |
| **Polar Congestion (LEO)** | 2,778 | 41.1% of LEO ⚠️ |

#### **Primary Mission Breakdown**
| Top Mission | Count | Share |
| :--- | :--- | :--- |
| **1. Communications** | 5,523 | 73.1% |
| **2. Earth Observation** | 1,310 | 17.3% |
| **3. Technology Development** | 441 | 5.8% |

#### **Data Quality & Imputation Success**
| Feature | Completeness | Method | Status |
| :--- | :--- | :--- | :--- |
| **Launch Mass Kg** | **100.0%** | Calculated/Imputed | ✅ SUCCESS |
| **Dry Mass Kg** | **100.0%** | Calculated/Imputed | ✅ SUCCESS |
| **Power Watts** | **100.0%** | Calculated/Imputed | ✅ SUCCESS |
| **Period Minutes** | **100.0%** | Calculated/Imputed | ✅ SUCCESS |
| **Lifetime Years** | **100.0%** | Calculated/Imputed | ✅ SUCCESS |
| **Perigee Km** | **100.0%** | Calculated/Imputed | ✅ SUCCESS |
| **Apogee Km** | **100.0%** | Calculated/Imputed | ✅ SUCCESS |
| **Eccentricity** | **100.0%** | Calculated/Imputed | ✅ SUCCESS |
| **Inclination Degrees** | **100.0%** | Calculated/Imputed | ✅ SUCCESS |
| **Geo Longitude** | **100.0%** | Calculated/Imputed | ✅ SUCCESS |

💾 **File Saved:** `../data/clean/ucs_cleaned.csv`

## **Registry Cleanup Complete**

**Summary of Operations:**
- **Normalized** 7,500+ active satellite entries into a standardized schema.
- **Reconstructed** missing physical data using a multi-tiered imputation engine (Keplerian physics, mass-ratios, and mission-based medians).
- **Engineered** Boolean flags for sector analysis and standardized orbital regimes.

**Next Notebook:** `satcat_cleanup.ipynb`
- Merge with the CelesTrak SATCAT to incorporate debris, rocket bodies, and radar cross-sections.